# Download Twitter Data

#### Import Dependencies

In [ ]:
# Tweepy library to connect to the Twitter API and process the response
import tweepy
# JSON to serialise JSON
import json
# Pandas to work with dataframes
import pandas as pd
# wget to retrieve profile images
import wget
# OS to manipulate file paths and names
import os
# Shutil to handle file duplicates
import shutil
# Time to handle API rate limiting
import time

# Read environment variables
from dotenv import load_dotenv

#### Load configuration file for environment variables

In [ ]:
load_dotenv('configuration.env')

#### Load Twitter authentication from environment variables

In [ ]:
# Twitter App credentials
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_ACCESS_TOKEN')
access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

#### Twitter Authentication

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3,retry_delay=5,retry_errors=set([401, 404, 500, 503]))

#### Set API Rate Limit

In [ ]:
rate_limit = 5

#### Seed User

In [ ]:
# User's screen name
screen_name = "BIM360"

#### Get User ID from Screen Name

In [ ]:
user = api.get_user(screen_name)
seed_user = user.id_str
print("User ID for " + screen_name + " is " + seed_user)

### Download Twitter Friends List

Function for downloading a list of a user's friends (the accounts they follow). The function requires the user ID of a 'seed' user.

**Note:** Twitter accounts marked 'private' or `protected` with return an empty list.

In [ ]:
def download_friends_list(user_id):
    
    friends = []
    
    try:
        # getting the friends list 
        friends = api.friends_ids(user_id, stringify_ids=True)
        
        print(user_id + " friends = " + str(len(friends)))
        
    except tweepy.TweepError as e: 
        print(user_id + " encountered Tweepy Error: {}".format(e))
        pass
    
    with open("./01_friends_lists/{filename}.json".format(filename=user_id), "w") as file:
        file.write(json.dumps(friends))
    
    return friends

#### Download friends of seed user

In [ ]:
friends = download_friends_list(seed_user)

#### Download friends-of-friends

If the download fails at any point you can restart from a specific point by taking a slice. Amending the first line of the following code to `for friend in friends[100:]:` will process all remaining items in the list from the hundredth element to the end.

In [ ]:
for friend in friends:
    
    fof = download_friends_list(str(friend))
    
    # Wait for rate limiter
    time.sleep(rate_limit)
        
print("Downloading friends-of-friends complete!")

### Identify Unique Users

This code identifies a list of unique users contained the friends lists captured above. Obtaining a list of unique users will speed up the process of downloading user details.

#### Set directory to read friends lists from

In [ ]:
data_directory = "./01_friends_lists/"

#### Identify unique users and calculate summary statistics

In [ ]:
filecount = 0
users = []
uniqueIDs = []

#Loop through files of friends lists
for file in os.listdir(data_directory):
    
    # Count files processed
    filecount += 1
    
    # Open the file
    with open(data_directory+file,encoding = "utf-8") as f:
        
        # Loop through friendlists
        for friendlist in f:
            
            # Convert JSON array to python list
            friends = json.loads(friendlist)
            
            # Add friends to list of users
            users = users + friends

#Create a dictionary with keys from users list to remove duplicates
d = dict.fromkeys(users)
#Unpack the dictionary back to a list
uniqueIDs = [*d]

print("---------SUMMARY---------")
print("Friends files = " + str(filecount))
print("Total users = " + str(len(users)))
print("Unique users = " + str(len(uniqueIDs)))

### Download Twitter User Details

Function for downloading public user information for each unique account identified in the friends list above. The function requires a list of user's IDs. It also accepts an optional optional argument `store_images=True` which will download a user's profile image or logo if they have one.

In [ ]:
def download_users(uniqueIDs, store_images=False):    
    
    # Count of processed users and batches
    user_count = 0
    batch_count = 0
    
    chunks = (len(uniqueIDs) - 1) // 100 + 1
    for i in range(chunks):
        batch = uniqueIDs[i*100:(i+1)*100]
    
        #Request users by ID in batches
        users = api.lookup_users(batch)
        
        # Increment friend count
        batch_count = batch_count + 1
        print("---------BATCH {}---------".format(str(batch_count)))

        for user in users:
            # Increment friend count
            user_count = user_count + 1
            print(str(user_count) + ": " + user.id_str + "_" + user.screen_name)

            #Store the user's data
            data = {"id_str": user.id_str,
                    "screen_name": user.screen_name,
                    "name": user.name,
                    "location": user.location,
                    "profile_image_url": user.profile_image_url,
                    "description": user.description,
                    # Placeholder for website url
                    "expanded_url": "",
                    "followers_count": user.followers_count,
                    "friends_count": user.friends_count,
                    "statuses_count": user.statuses_count,
                    # Format datetime object
                    "created_at": user.created_at.strftime("%y-%m-%d, %H:%M:%S"),
                    "protected": user.protected,
                    "verified": user.verified
                   }

            # Check for website URL and add to data
            if ("url" in user.entities):
                data["expanded_url"] = user.entities["url"]["urls"][0]["expanded_url"],
            else:
                print("No associated url")

            # Save user details
            with open("./02_user_details/{twitter_id}.json".format(twitter_id=user.id_str), "w") as file:
                file.write(json.dumps(data))

            # Save the Twitter profile image
            if store_images == True:
                file_destination = os.path.join("03_logos/" + user.screen_name + ".png")
                # Check if file already exists
                if os.path.exists(file_destination) != True:
                    try:
                        # Remove the '_normal' tag from the URL to get a full sized image
                        link = user.profile_image_url.replace("_normal","")
                        # Use wget to download and save the image file
                        wget.download(link, file_destination)
                    except:
                        print("Profile image not available")
                else:
                    print("Profile image already downloaded")
            else:
                pass

        # Wait for rate limiter
        time.sleep(rate_limit)    
    
    print("Processing complete!")
    return users

#### Download User Details

If the download fails at any point you can restart from a specific point by taking a slice. Amending the first line of the following code to for `uniqueIDs[100:]:` will process all remaining items in the list from the hundredth element to the end.

In [ ]:
output = download_users(uniqueIDs, store_images=True)